# Goal:

For the sales, I'd like to denote whether it was sold at a loss or a gain with a red or green icon. The rest of the info could be put into a pop up box, if that works

oh and btw maybe put listings in blue or something so we can see that they are not sold yet

# Imports

In [1]:
import pandas as pd
import geopandas as gpd
import googlemaps
from shapely.geometry import Point
import os
import numpy as np
import re
from textwrap import shorten
from bs4 import BeautifulSoup
import gspread

## Data read-in

In [2]:
df = pd.read_csv('NYC_Office_to_Resi_FOR_MAP - Sheet1.csv')

In [3]:
def create_hyperlink_limited(description, url, word_limit=5):
    """Create an HTML anchor tag for the first few words of the description with the URL."""
    if pd.notnull(url):  # Check if there's a valid URL
        words = description.split()  # Split the description into words
        if len(words) > word_limit:  # Only wrap the first word_limit words
            first_part = ' '.join(words[:word_limit])
            remaining_part = ' '.join(words[word_limit:])
            return f'<a href="{url}">{first_part}</a> {remaining_part}'
        else:
            return f'<a href="{url}">{description}</a>'  # If fewer than word_limit words, wrap the whole text
    return description  # Return original description if no URL

# Apply the function to combine 'Description' and 'URL'
df['Description'] = df.apply(lambda row: create_hyperlink_limited(row['Description'], row['URL'], word_limit=5), axis=1)

In [4]:
# Drop the 'URL' column if you no longer need it
df = df.drop(columns=['URL'])

In [5]:
df['Description'].iloc[0]

'The former Pfizer headquarters is being transformed by Metro Loft and architect Gensler into a 29-story mixed-use building with 1,463 rental apartments, marking a major Midtown East residential addition.'

## Geocode

In [6]:
df['Neighborhood'] = 'NYC'

In [7]:
df['full_address'] = df['Address'] + ' ' + df['Neighborhood'] + ' ' + df['State']

In [8]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [9]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'Full_Address' column and store the results in the 'geocoded' column
df['geocoded'] = df['full_address'].apply(geocode)

# Split 'geocoded' into 'lat' and 'lon', handling None values
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x is None else x).apply(pd.Series)

# Convert 'lat' and 'lon' to floats
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

# Create the 'geometry' column with Point objects
df['geometry'] = df.apply(lambda row: Point(row['lon'], row['lat']) if pd.notnull(row['lat']) and pd.notnull(row['lon']) else None, axis=1)

# Drop rows where geometry is None
df = df.dropna(subset=['geometry'])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set CRS (Coordinate Reference System) to WGS 84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)


/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:131: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,Address,Description,Neighborhood,State,full_address,geocoded,lat,lon,geometry
0,219-229 East 42nd Street,The former Pfizer headquarters is being transf...,NYC,New York,219-229 East 42nd Street NYC New York,"(40.75045230000001, -73.9729928)",40.750452,-73.972993,POINT (-73.97299 40.75045)
1,25 Water Street,"GFP, Metro Loft Management and Rockwood Capita...",NYC,New York,25 Water Street NYC New York,"(40.70293179999999, -74.010592)",40.702932,-74.010592,POINT (-74.01059 40.70293)
2,212 Broadway,"<a href=""https://therealdeal.com/new-york/2024...",NYC,New York,212 Broadway NYC New York,"(40.86847849999999, -73.91859749999999)",40.868478,-73.918597,POINT (-73.91860 40.86848)
3,49 Broad Street,Silverstein Properties and Metro Loft bought t...,NYC,New York,49 Broad Street NYC New York,"(40.7054115, -74.0111474)",40.705411,-74.011147,POINT (-74.01115 40.70541)
4,110 Maiden Lane,"<a href=""https://therealdeal.com/new-york/2024...",NYC,New York,110 Maiden Lane NYC New York,"(40.7063748, -74.0066279)",40.706375,-74.006628,POINT (-74.00663 40.70637)
5,330 West 42nd Street,"<a href=""https://therealdeal.com/new-york/2023...",NYC,New York,330 West 42nd Street NYC New York,"(40.7575608, -73.99162969999999)",40.757561,-73.991630,POINT (-73.99163 40.75756)
6,175 Pearl Street,"<a href=""https://therealdeal.com/new-york/2024...",NYC,New York,175 Pearl Street NYC New York,"(40.7062045, -74.0074501)",40.706204,-74.007450,POINT (-74.00745 40.70620)
7,830 Third Avenue,"<a href=""https://therealdeal.com/new-york/2022...",NYC,New York,830 Third Avenue NYC New York,"(40.80757560000001, -73.9311385)",40.807576,-73.931139,POINT (-73.93114 40.80758)
8,650 First Avenue,"<a href=""https://therealdeal.com/new-york/2023...",NYC,New York,650 First Avenue NYC New York,"(40.7458861, -73.9717258)",40.745886,-73.971726,POINT (-73.97173 40.74589)
9,1751 Park Avenue,Yitzchok Joseph of 1163 Fulton Realty LLC is b...,NYC,New York,1751 Park Avenue NYC New York,"(40.8026899, -73.9403842)",40.802690,-73.940384,POINT (-73.94038 40.80269)


In [10]:
gdf

,Address,Description,Neighborhood,State,full_address,geocoded,lat,lon,geometry
0,219-229 East 42nd Street,The former Pfizer headquarters is being transf...,NYC,New York,219-229 East 42nd Street NYC New York,"(40.75045230000001, -73.9729928)",40.750452,-73.972993,POINT (-73.97299 40.75045)
1,25 Water Street,"GFP, Metro Loft Management and Rockwood Capita...",NYC,New York,25 Water Street NYC New York,"(40.70293179999999, -74.010592)",40.702932,-74.010592,POINT (-74.01059 40.70293)
2,212 Broadway,"<a href=""https://therealdeal.com/new-york/2024...",NYC,New York,212 Broadway NYC New York,"(40.86847849999999, -73.91859749999999)",40.868478,-73.918597,POINT (-73.91860 40.86848)
3,49 Broad Street,Silverstein Properties and Metro Loft bought t...,NYC,New York,49 Broad Street NYC New York,"(40.7054115, -74.0111474)",40.705411,-74.011147,POINT (-74.01115 40.70541)
4,110 Maiden Lane,"<a href=""https://therealdeal.com/new-york/2024...",NYC,New York,110 Maiden Lane NYC New York,"(40.7063748, -74.0066279)",40.706375,-74.006628,POINT (-74.00663 40.70637)
5,330 West 42nd Street,"<a href=""https://therealdeal.com/new-york/2023...",NYC,New York,330 West 42nd Street NYC New York,"(40.7575608, -73.99162969999999)",40.757561,-73.991630,POINT (-73.99163 40.75756)
6,175 Pearl Street,"<a href=""https://therealdeal.com/new-york/2024...",NYC,New York,175 Pearl Street NYC New York,"(40.7062045, -74.0074501)",40.706204,-74.007450,POINT (-74.00745 40.70620)
7,830 Third Avenue,"<a href=""https://therealdeal.com/new-york/2022...",NYC,New York,830 Third Avenue NYC New York,"(40.80757560000001, -73.9311385)",40.807576,-73.931139,POINT (-73.93114 40.80758)
8,650 First Avenue,"<a href=""https://therealdeal.com/new-york/2023...",NYC,New York,650 First Avenue NYC New York,"(40.7458861, -73.9717258)",40.745886,-73.971726,POINT (-73.97173 40.74589)
9,1751 Park Avenue,Yitzchok Joseph of 1163 Fulton Realty LLC is b...,NYC,New York,1751 Park Avenue NYC New York,"(40.8026899, -73.9403842)",40.802690,-73.940384,POINT (-73.94038 40.80269)


In [11]:
# Convert all data to strings if possible, or remove complex structures
for col in gdf.columns:
    if gdf[col].dtype == object:
        try:
            gdf[col] = gdf[col].astype(str)
        except Exception as e:
            print(f"Could not convert column {col}: {e}")
            gdf.drop(columns=[col], inplace=True)


In [12]:
gdf.to_file('data.geojson', driver='GeoJSON')

In [13]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/NYC_Office_to_Resi_10_21_24


In [14]:
df.columns

Index(['Address', 'Description', 'Neighborhood', 'State', 'full_address',
       'geocoded', 'lat', 'lon', 'geometry'],
      dtype='object')

<a href="https://therealdeal.com/chicago/2024/04/03/david-gassman-sells-chicago-apartment-portfolio-for-72m/"> 6954 North Sheridan Road, Chicago</a>

In [15]:
gdf.columns

Index(['Address', 'Description', 'Neighborhood', 'State', 'full_address',
       'geocoded', 'lat', 'lon', 'geometry'],
      dtype='object')